In [20]:
import os
import numpy as np

In [21]:
os.makedirs("final_dataset/train/image", exist_ok=True)
os.makedirs("final_dataset/train/segmentation", exist_ok=True)
os.makedirs("final_dataset/valid/image", exist_ok=True)
os.makedirs("final_dataset/valid/segmentation", exist_ok=True)
os.makedirs("final_dataset/test/image", exist_ok=True)
os.makedirs("final_dataset/test/segmentation", exist_ok=True)

In [22]:
import json
json_file_path = 'data_npy/depth_lists.json'

depth_dict = {}
with open(json_file_path, 'r') as json_file:
    depth_dict = json.load(json_file)

In [23]:
def find_non_kidneys(root_path, depth_dict, kind):
    file_path_list = []
    for file_path in sorted(os.listdir(root_path)):
        file_path_list.append(os.path.join(root_path, file_path))

    count_slices = 0
    removing_list = set()
    for depth in depth_dict[kind]:
        count_no_kidney_slices = 0
        for i in range(depth):
            inx = count_slices + i
            file_path = file_path_list[inx]
            arr = np.load(file_path)[1]
            
            if np.max(arr) == 0:
                count_no_kidney_slices += 1
        #########################################
        no_not_checking = count_no_kidney_slices // 2        
        flag_counter = 0
        for i in range(depth):
            inx = count_slices + i
            file_path = file_path_list[inx]
            arr = np.load(file_path)[1]
            
            if flag_counter > no_not_checking:
                break
            else:
                if np.max(arr) == 0:
                    flag_counter += 1
                    removing_list.add(file_path.split('/')[-1])
            
            if np.max(arr) == 0:
                count_no_kidney_slices += 1
        count_slices += depth
    return removing_list

import shutil

def create_new_dataset(root_path, destination_folder, removing_list):
    count = 0
    for file_path in sorted(os.listdir(root_path)):
        if file_path not in removing_list:
            count += 1
            source_folder = os.path.join(root_path, file_path)
            shutil.copy(source_folder, destination_folder)
    print(count)

In [24]:
removing_list_train = find_non_kidneys(root_path='data_npy/train/segmentation', depth_dict=depth_dict, kind='train')
removing_list_valid = find_non_kidneys(root_path='data_npy/valid/segmentation', depth_dict=depth_dict, kind='valid')
removing_list_test = find_non_kidneys(root_path='data_npy/test/segmentation', depth_dict=depth_dict, kind='test')

In [25]:
create_new_dataset(root_path='data_npy/train/image/', destination_folder='final_dataset/train/image/', removing_list=removing_list_train)
create_new_dataset(root_path='data_npy/valid/image/', destination_folder='final_dataset/valid/image/', removing_list=removing_list_valid)
create_new_dataset(root_path='data_npy/test/image/', destination_folder='final_dataset/test/image/', removing_list=removing_list_test)

create_new_dataset(root_path='data_npy/train/segmentation/', destination_folder='final_dataset/train/segmentation/', removing_list=removing_list_train)
create_new_dataset(root_path='data_npy/valid/segmentation/', destination_folder='final_dataset/valid/segmentation/', removing_list=removing_list_valid)
create_new_dataset(root_path='data_npy/test/segmentation/', destination_folder='final_dataset/test/segmentation/', removing_list=removing_list_test)

10389
1964
1344
10389
1964
1344


In [26]:
len_train = len(os.listdir('final_dataset/train/image/'))
len_valid = len(os.listdir('final_dataset/train/image/'))
len_test = len(os.listdir('final_dataset/train/image/'))

In [32]:
def renamer(kind):
    for i, file_path in enumerate(sorted(os.listdir(f'final_dataset/{kind}/image/'))):
        num =  '{:05d}'.format(i)
        os.rename(os.path.join(f'final_dataset/{kind}/image/', file_path), os.path.join(f'final_dataset/{kind}/image/', num+'.npy')) 
        os.rename(os.path.join(f'final_dataset/{kind}/segmentation/', file_path), os.path.join(f'final_dataset/{kind}/segmentation/', num+'.npy'))
renamer('train')
renamer('valid')
renamer('test')

In [35]:
print(len(os.listdir('kits19/train/image')))

37250


In [34]:
import monai
from monai.networks.nets.segresnet import SegResNet

SegResNet_model = SegResNet(
    spatial_dims=2,
    init_filters=16,
    in_channels=1,
    out_channels=2,
    dropout_prob=0.2,
)
print(SegResNet_model)

SegResNet(
  (act_mod): ReLU(inplace=True)
  (convInit): Convolution(
    (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (down_layers): ModuleList(
    (0): Sequential(
      (0): Identity()
      (1): ResBlock(
        (norm1): GroupNorm(8, 16, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 16, eps=1e-05, affine=True)
        (act): ReLU(inplace=True)
        (conv1): Convolution(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (conv2): Convolution(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
    )
    (1): Sequential(
      (0): Convolution(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (1): ResBlock(
        (norm1): GroupNorm(8, 32, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 32, eps=1e-05, affine=True)
        (act): ReLU(inpl